In [10]:
import aiohttp
import asyncio
import numpy as np
import ujson
from multiprocessing.dummy import Pool as ThreadPool 

In [11]:
class ModelClient():
    
    #MODEL_HOST = 'http://192.168.64.8:30123'
    MODEL_HOST = 'http://10.100.241.46:8501'
    
    def __init__(self, model_host = MODEL_HOST):
        self.model_host = model_host
            
    async def async_predict(self, indata):    
        async with aiohttp.ClientSession() as session:
            async with session.post(self.model_host + '/v1/models/model:predict', json=self.numpy_to_json(indata)) as resp:
                response_json = await resp.json()
                return self.json_to_numpy(response_json)
    
    def predict(self, indata):
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.async_predict(indata))
    
    def json_to_numpy(self, json_str, key='predictions'):
        return np.array(json_str[key])

    def numpy_to_json(self, nd_array, key='instances'):
        try:
            if nd_array.shape[1] == 1:
                nd_array.reshape(nd_array.shape[0])
            nd_array_list =  { key : nd_array.tolist() } 
            return nd_array_list
        except Exception as e:
            nd_array_list =  { key : [] } 
            return nd_array_list


In [12]:
modelClient = ModelClient()
indata = np.random.randint(100000, size=(5000, 20)).astype(np.float32)


indata_batches = [indata, indata, indata, indata, indata, indata, indata, indata, indata, indata,
                 indata, indata, indata, indata, indata, indata, indata, indata, indata, indata,
                 indata, indata, indata, indata, indata, indata, indata, indata, indata, indata,
                 indata, indata, indata, indata, indata, indata, indata, indata, indata, indata,
                 indata, indata, indata, indata, indata, indata, indata, indata, indata, indata]

In [15]:
def make_prediction(indata):
    #print("Request", indata.shape)
    result = modelClient.predict(indata)
    #print("Response", result.shape)

In [17]:
pool = ThreadPool(20) 

%time result = pool.map(make_prediction, indata_batches)

CPU times: user 3.05 s, sys: 271 ms, total: 3.33 s
Wall time: 5.02 s


In [9]:
250000 / 4.44

56306.3063063063